In [0]:




# Databricks Notebook: Komplett ETL-pipeline for parkeringsdata (uten Unity Catalog)

# ------------------------------
# 1. Hent data fra ekstern URL
# ------------------------------
import requests
import json
from pyspark.sql import SparkSession
from etl_pipeline import sjekk_duplikater, valider_manglende, valider_gyldige_verdier, konverter_timestamp

# Hent JSON-data fra en offentlig URL
url = "https://opencom.no/dataset/36ceda99-bbc3-4909-bc52-b05a6d634b3f/resource/d1bdc6eb-9b49-4f24-89c2-ab9f5ce2acce/download/parking.json"
response = requests.get(url)
data = response.json()

# Konverter JSON-data til Spark DataFrame
# (Datatypene blir automatisk inferert, men kan tilpasses manuelt ved behov)
df_raw = spark.createDataFrame(data)

valider_manglende(df_raw)
valider_gyldige_verdier(df_raw)
df_deduped = sjekk_duplikater(df_raw)
df_cleaned = konverter_timestamp(df_deduped)


# --- Datakvalitetssjekker ---
from pyspark.sql.functions import col, isnull, count, when
import logging

# Konfigurer logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Sjekk for duplikater basert på primærnøkkelkandidater (f.eks. Sted + Dato + Klokkeslett)
# Må gjøres FØR konvertering til timestamp for å bevare originalitet
df_raw_with_id = df_raw.withColumn(
    "unique_id",
    concat_ws("_", col("Sted"), col("Dato"), col("Klokkeslett"))
)
duplicate_count = df_raw_with_id.groupBy("unique_id").agg(count("*").alias("count")).filter(col("count") > 1).count()

if duplicate_count > 0:
    logger.warning(f"ADVARSEL: {duplicate_count} duplikate rader funnet i rådataene basert på Sted, Dato, Klokkeslett.")
    # Du kan velge å filtrere bort duplikater her, f.eks.:
    df_raw = df_raw_with_id.dropDuplicates(["Sted", "Dato", "Klokkeslett"]).drop("unique_id")
    logger.info("Duplikater fjernet.")
else:
    logger.info("Ingen duplikater funnet i rådataene.")
    df_raw = df_raw_with_id.drop("unique_id") # Fjern midlertidig kolonne hvis ingen duplikater

# Sjekk for manglende nøkkelverdier (f.eks. Sted, Dato, Klokkeslett, Antall_ledige_plasser)
for column_name in ["Sted", "Dato", "Klokkeslett", "Antall_ledige_plasser"]:
    missing_count = df_raw.filter(isnull(col(column_name))).count()
    if missing_count > 0:
        logger.error(f"FEIL: {missing_count} rader har manglende verdier i '{column_name}' kolonnen.")
        # Avhengig av din policy, kan du:
        # 1. Droppe rader med manglende nøkkelverdier:
        # df_raw = df_raw.na.drop(subset=[column_name])
        # logger.info(f"Rader med manglende verdier i '{column_name}' er fjernet.")
        # 2. Fylle med en standardverdi:
        # df_raw = df_raw.fillna({column_name: "UKJENT"})
        # 3. Kaste en feil og stoppe pipeline (anbefalt for kritiske feil):
        raise ValueError(f"Kritisk feil: Manglende verdier i '{column_name}'. Stopper pipeline.")
    else:
        logger.info(f"Ingen manglende verdier funnet i '{column_name}' kolonnen.")

# Sjekk for korrekte datatyper (Spark infererer, men det kan være lurt å verifisere)
# For 'Antall_ledige_plasser', sjekk at den er numerisk og ikke negativ
try:
    # Kast en feil hvis det er ikke-numeriske verdier som ikke kan castes
    # Dette er mer robust hvis du forventer integer/long
    df_raw.select(col("Antall_ledige_plasser").cast("int")).collect()
    negative_count = df_raw.filter(col("Antall_ledige_plasser") < 0).count()
    if negative_count > 0:
        logger.warning(f"ADVARSEL: {negative_count} rader har negative verdier i 'Antall_ledige_plasser'.")
        raise ValueError(f"Kritisk feil: 'Antall_ledige_plasser' inneholder negative verdier. Stopper pipeline.")
    else:
        logger.info("Alle 'Antall_ledige_plasser' er gyldige (ikke negative).")

except Exception as e:
    logger.error(f"FEIL: 'Antall_ledige_plasser' kan ikke konverteres til numerisk type. Detaljer: {e}")
    raise TypeError(f"Kritisk feil: 'Antall_ledige_plasser' er ikke numerisk. Stopper pipeline.")

# Fortsett med rensing og transformasjon kun hvis datakvaliteten er akseptabel
# df_cleaned = df_raw.withColumn(...)

# ------------------------------------------
# 2. Rens og konverter dato/klokkeslett
# ------------------------------------------
from pyspark.sql.functions import to_timestamp, concat_ws

# Lag en timestamp-kolonne fra Dato og Klokkeslett, og fjern de gamle kolonnene
df_cleaned = df_raw.withColumn(
    "timestamp",
    to_timestamp(concat_ws(" ", df_raw["Dato"], df_raw["Klokkeslett"]), "dd.MM.yyyy HH:mm")
).drop("Dato", "Klokkeslett")

from pyspark.sql.functions import count




# 3. Lagre staging-tabell (bronse)
# ----------------------------------
# Append for å bevare historiske data (bronse = ubehandlet rådata)
bronze_path = "/mnt/bronze/staging_parking"
df_cleaned.write.format("delta").mode("append").option("mergeSchema", "true").save(bronze_path)
spark.sql(f"CREATE TABLE IF NOT EXISTS default.staging_parking USING DELTA LOCATION '{bronze_path}'")

# ----------------------------------
# 4. Dimensjonstabell: Parkering
# ----------------------------------
from pyspark.sql.functions import col

# Hent eksisterende tabell hvis den finnes, ellers None
existing_dim_parkering = spark.table("default.dim_parkering") if "default.dim_parkering" in [t.name for t in spark.catalog.listTables("default")] else None

# Finn nye unike parkeringsplasser og gi kolonnene mer beskrivende navn
new_dim_parkering = df_cleaned.select("Sted", "Latitude", "Longitude").distinct()
new_dim_parkering = new_dim_parkering.withColumnRenamed("Sted", "Parkering_navn")

# Vi antar at det alltid skal være nøyaktig 9 parkeringsplasser. Skriv bare hvis antallet ikke stemmer.
if existing_dim_parkering is None or existing_dim_parkering.count() != 9:
    new_dim_parkering.write.format("delta").mode("overwrite").saveAsTable("default.dim_parkering")

# ----------------------------------
# 5. Dimensjonstabell: Tid
# ----------------------------------
from pyspark.sql.functions import to_date, hour, minute

# Trekk ut dato, time og minutt fra timestamp
dim_tid_batch = df_cleaned.select(
    to_date("timestamp").alias("dato"),
    hour("timestamp").alias("time"),
    minute("timestamp").alias("minutt")
).distinct()

# Skriv kun nye tidspunkt til tabellen
if "default.dim_tid" in [t.name for t in spark.catalog.listTables("default")]:
    existing_dim_tid = spark.table("default.dim_tid")
    delta_tid = dim_tid_batch.subtract(existing_dim_tid)
    if delta_tid.count() > 0:
        delta_tid.write.format("delta").mode("append").saveAsTable("default.dim_tid")
else:
    dim_tid_batch.write.format("delta").mode("overwrite").saveAsTable("default.dim_tid")

# ----------------------------------
# 6. Faktatabell: Parkeringskapasitet
# ----------------------------------
# Hent 9 observasjoner per kjøring (én per lokasjon)
new_fakt = df_cleaned.select(
    to_date("timestamp").alias("dato"),
    "Sted",
    "Antall_ledige_plasser",
    "timestamp"
)

# Append for å lagre hver kjørings observasjoner
new_fakt.write.format("delta").mode("append").saveAsTable("default.fakt_parkering")

# ----------------------------------
# 7. Visualisering: Utvikling per lokasjon (én dag)
# ----------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates

# --- Hent full historikk fra faktatabellen ---
df_plot = spark.table("default.fakt_parkering")

# Konverter til Pandas
df_pd = df_plot.toPandas()

# --- Konverter timestamp til datetime ---
df_pd['datetime'] = pd.to_datetime(df_pd['timestamp'])

# --- Velg én dag for visualisering ---
valgt_dato = "2025-06-11"
df_pd['dato'] = df_pd['datetime'].dt.date
df_plot_dag = df_pd[df_pd['dato'] == pd.to_datetime(valgt_dato).date()]

# Sjekk om vi har noen data
unike_steder = df_plot_dag['Sted'].unique()

if len(unike_steder) > 0:
    fig, axs = plt.subplots(len(unike_steder), 1, figsize=(12, 4 * len(unike_steder)), sharex=True)

    if len(unike_steder) == 1:
        axs = [axs]

    for ax, sted in zip(axs, unike_steder):
        subset = df_plot_dag[df_plot_dag['Sted'] == sted].sort_values('datetime')
        ax.plot(subset['datetime'], subset['Antall_ledige_plasser'], marker='o', linestyle='-')
        ax.set_title(f"Ledige plasser - {sted}")
        ax.set_ylabel("Antall ledige")
        ax.grid(True)


    axs[-1].xaxis.set_major_locator(mdates.AutoDateLocator())
    axs[-1].xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    plt.xlabel("Tid (klokkeslett)")
    fig.autofmt_xdate(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Ingen data funnet for valgt dato:", valgt_dato)
